In [37]:
import sys
sys.path.append('../')
import torch
from tools.utils import manual_seed
import PIL
import PIL.Image

import numpy as np
import matplotlib.pyplot as plt
from tools.img import rgb2yuv, yuv2rgb, rgb_to_yuv, yuv_to_rgb
from torchvision.transforms.transforms import ToTensor
from tools.dataset import get_dataloader, get_dataset_class_and_scale, get_dataset_normalization, get_de_normalization
from omegaconf import OmegaConf
from tools.img import tensor2ndarray, ndarray2tensor
from torchmetrics import StructuralSimilarityIndexMeasure
from tools.dataset import get_dataloader
from tools.inject_backdoor import patch_trigger

manual_seed(42)

In [38]:
train_dl, test_dl = get_dataloader('cifar10', 64, False, 8)
x_c = None
y_c = None
for batch, label in test_dl:
    batch = get_de_normalization('cifar10')(batch)
    for i in range(batch.shape[0]):
        if label[i] == 3:
            x_c = batch[i]
            y_c = label[i]
            break
    if x_c != None:
        break

x_p = x_c.clone().to(torch.float64)

x_p_yuv = torch.stack(rgb_to_yuv(x_p[0], x_p[1], x_p[2]), dim=0)
x_p_rgb = torch.stack(yuv_to_rgb(x_p_yuv[0], x_p_yuv[1], x_p_yuv[2]))


(x_c - x_p_rgb).mean()


tensor(8.4538e-07, dtype=torch.float64)